<a href="https://colab.research.google.com/github/HamdanXI/nlp_adventure/blob/main/802%20code/bert-paradetox-1token-mask-evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets transformers

In [11]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch

# Load the dataset
dataset = load_dataset("HamdanXI/paradetox-1Token-Split-MASK")

# Load the tokenizer and model
model_name = "HamdanXI/bert-base-uncased-paradetox-1Token-Split-MASK-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)

# Function to predict the masked token
def predict_masked_token(sentence):
    inputs = tokenizer(sentence, return_tensors="pt")
    mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]

    token_logits = model(**inputs).logits
    mask_token_logits = token_logits[0, mask_token_index, :]
    top_token = torch.argmax(mask_token_logits).item()

    predicted_token = tokenizer.decode([top_token])
    return predicted_token

predicted_sentences = []

# Iterate over the test split and predict
for row in dataset['test']:
    sentence_with_mask = row['input']
    predicted_token = predict_masked_token(sentence_with_mask)
    predicted_sentence = sentence_with_mask.replace(tokenizer.mask_token, predicted_token)
    predicted_sentences.append(predicted_sentence)

In [ ]:
!pip install bert-score
from bert_score import score

In [17]:
def bert_score_evaluate(predictions, label):
  # Compute BERT Score
  P, R, F1 = score(predictions, label, lang="en", rescale_with_baseline=True)

  # Compute and print the average scores
  print(f"Precision: {P.mean()}, Recall: {R.mean()}, F1 Score: {F1.mean()}")

In [18]:
bert_score_evaluate(predicted_sentences, dataset['test']['labels'])

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Precision: 0.8262403011322021, Recall: 0.860024094581604, F1 Score: 0.8430630564689636


In [ ]:
!pip install git+https://github.com/google-research/bleurt.git
from datasets import load_metric

bleurt_metric = load_metric('bleurt')

def bleurt_evaluate(predictions, label):
    results = bleurt_metric.compute(predictions=predictions, references=label)

    scores = results['scores']

    average_score = sum(scores) / len(scores) if scores else 0
    print(f"Average BLEURT Score: {average_score}")

In [20]:
bleurt_evaluate(predicted_sentences, dataset['test']['labels'])

Average BLEURT Score: 0.535543645969816


In [ ]:
!pip install evaluate
!pip install unbabel-comet

from evaluate import load
comet_metric = load('comet')

In [25]:
def comet_evaluate(predictions, input, label):
    comet_score = comet_metric.compute(predictions=predictions, references=label, sources=input)
    print(comet_score)

In [ ]:
comet_evaluate(predicted_sentences, dataset['test']['input'], dataset['test']['labels'])

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
